In [0]:
!pip install tensorflow


**Chapter 3**

**Problem 1** Imdb review classification

In [0]:
#Binary classification with IMDB movie review dataset

from keras.datasets import imdb
from keras import models
from keras import layers
import numpy as np

(Xtrain,Ytrain),(Xtest,Ytest)= imdb.load_data(num_words=10000)

def vectorize(sequences,dimensions=10000):
  tensor=np.zeros((len(sequences),dimensions))
  for i,sequence in enumerate(sequences):
    tensor[i,sequence]=1
  return tensor
tensor=vectorize(Xtrain)
Xtrain=tensor
Xtest=vectorize(Xtest)
Ytrain=np.asarray(Ytrain).astype('float32')
Ytest=np.asarray(Ytest).astype('float32')

Xval=Xtrain[:10000]
new_Xtrain=Xtrain[10000:]
Yval=Ytrain[:10000]
new_Ytrain=Ytrain[10000:]

model=models.Sequential()
model.add(layers.Dense(16,activation='relu',input_shape=(10000,)))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(new_Xtrain,new_Ytrain,epochs=20,batch_size=512,validation_data=(Xval,Yval))
model.evaluate(Xtest,Ytest,batch_size=512)



**Problem 2** 
News category prediction



In [0]:
from keras.datasets import reuters
import numpy as np
from keras import models
from keras import layers
(Xtrain,Ytrain),(Xtest,Ytest)=reuters.load_data(num_words=10000)
def vectorize(sequences,dimensions=10000):
  tensor=np.zeros((len(sequences),dimensions))
  for i,sequence in enumerate(sequences):
    tensor[i,sequence]=1
  return tensor
Xtrain=vectorize(Xtrain)
Xtest=vectorize(Xtest)
Ytest= np.asarray(Ytest).astype('float32')
Ytrain=np.asarray(Ytrain).astype('float32')
Xval=Xtrain[:2000]
Yval=Ytrain[:2000]
Xtrain=Xtrain[2000:]
Ytrain=Ytrain[2000:]

model= models.Sequential()
model.add(layers.Dense(64,activation='relu',input_shape=(10000,)))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(46,activation='softmax'))
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(Xtrain,Ytrain,epochs=20,batch_size=512,validation_data=[Xval,Yval])
model.evaluate(Xtest,Ytest)



**Problem 3**: Housing price prediction

In [0]:
from keras.datasets import boston_housing
from keras import models
from keras import layers
import numpy as np

(Xtrain,Ytrain),(Xtest,Ytest)=boston_housing.load_data()
Xmean=Xtrain.mean(axis=0)
Xtrain-=Xmean
Xtest-=Xmean
Xstd=Xtrain.std(axis=0)
Xtrain/=Xstd
Xtest/=Xstd

def build_model():
  model=models.Sequential()
  model.add(layers.Dense(64,activation='relu',input_shape=(Xtrain.shape[1],)))
  model.add(layers.Dense(64,activation='relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
  return model

k=4
num_segments=len(Xtrain)//k
model_outputs=[]

for i in range(k):
  val_x=Xtrain[num_segments*i:num_segments*(i+1)]
  val_y=Ytrain[num_segments*i:num_segments*(i+1)]
  Xtrain_set= np.concatenate([Xtrain[:num_segments*i],Xtrain[num_segments*(i+1):]],axis=0)
  Ytrain_set=np.concatenate([Ytrain[:num_segments*i],Ytrain[num_segments*(i+1):]],axis=0)
  model=build_model()
  model.fit(Xtrain_set,Ytrain_set,epochs=100,batch_size=1)
  mse,mae=model.evaluate(val_x,val_y)
  model_outputs.append(mae)
model_mean=np.mean(model_outputs)
  




**Chapter 5**

**Problem 1 (Cats and Dogs classification)**

Training a network from scratch

In [0]:
#Setting up the kaggle api to download datasets directly from kaggle
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list
!kaggle competitions download -c dogs-vs-cats

#arranging the downloaded dataset into train, test, and validation file folders
!unzip train.zip 
!unzip test1.zip
!rm train.zip
!rm test1.zip
!mkdir train_set
!mkdir test_set
%cd train_set
!mkdir train_dog
!mkdir train_cat
%cd ..
%cd test_set
!mkdir test_dog
!mkdir test_cat
%cd ..
!mkdir val_set
%cd val_set
!mkdir val_cat
!mkdir val_dog
%cd ..
import shutil
fnames_cat =['cat.{}.jpg'.format(i) for i in range(1000) ]
for i in fnames_cat:
  shutil.copy('train/'+i,'train_set/train_cat')
fnames_dog=['dog.{}.jpg'.format(i) for i in range(1000)]
for i in fnames_dog:
  shutil.copy('train/'+i,'train_set/train_dog')
fnames_cat =['cat.{}.jpg'.format(i) for i in range(1001,1501) ]
for i in fnames_cat:
  shutil.copy('train/'+i,'test_set/test_cat')
fnames_dog =['dog.{}.jpg'.format(i) for i in range(1001,1501) ]
for i in fnames_dog:
  shutil.copy('train/'+i,'test_set/test_dog')
fnames_cat =['cat.{}.jpg'.format(i) for i in range(1502,2002) ]
for i in fnames_cat:
  shutil.copy('train/'+i,'val_set/val_cat')
fnames_dog =['dog.{}.jpg'.format(i) for i in range(1502,2002) ]
for i in fnames_dog:
  shutil.copy('train/'+i,'val_set/val_dog')

#Preprocessing the image and convert it into a tensor format so that the neural network can accept it as an input
from keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(rescale=1./255)
traingen=datagen.flow_from_directory('train_set',target_size=(150,150),batch_size=20,class_mode='binary')
testgen=datagen.flow_from_directory('test_set',target_size=(150,150),batch_size=20,class_mode='binary')
valgen=datagen.flow_from_directory('val_set',target_size=(150,150),batch_size=20,class_mode='binary') 


#building the model
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
def build_model():
  model=models.Sequential()
  model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Conv2D(64,(3,3),activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Conv2D(128,(3,3),activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Conv2D(128,(3,3),activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(512,activation='relu'))
  model.add(layers.Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

#training the model, saving and downloading the model file
model=build_model()
model.fit_generator(traingen,epochs=30,validation_data=valgen,steps_per_epoch=100,validation_steps=50)
model.save('cats_vs_dog.h5')  
files.download('cats_vs_dog.h5')

Using a pretrained network


In [0]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras import models,layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

conv_base=VGG19(weights='imagenet',include_top=False,input_shape=(150,150,3))
conv_base.trainable=False
model=models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
datagen=ImageDataGenerator(rescale=1./255,rotation_range=40,height_shift_range=0.2,width_shift_range=0.2,shear_range=0.2,horizontal_flip=True,fill_mode='nearest')
traingen= datagen.flow_from_directory('train_set',target_size=(150,150),batch_size=20,class_mode='binary')
valgen= datagen.flow_from_directory('val_set',target_size=(150,150),batch_size=20,class_mode='binary')
model.fit_generator(traingen,steps_per_epoch=100,epochs=30,validation_data=valgen,validation_steps=20)




Chapter 6

**Word level one hot encoding**

In [0]:
import numpy as np
samples=['the cat sat on a mat','He took the dog for a walk']

sample_dict={}
for sample in samples:
  for word in sample.split(" "):
    if word not in sample_dict.keys():
      sample_dict[word]=len(sample_dict)+1

max_len=10

results=np.zeros((len(samples),max_len,max(sample_dict.values())+1))
for i,sample in enumerate(samples):
  for j,words in list(enumerate(sample.split(" ")))[:max_len]:
    index=sample_dict.get(words)
    print(words,index)
    results[i,j,index]=1




**Character level One Hot Encoding**

In [0]:
import string
samples=['the cat sat on a mat','He took the dog for a walk']

characters= string.printable
token_index=dict(zip(range(1,len(characters)+1),characters))
max_len=50

results=np.zeros((len(samples),max_len,len(token_index.keys())+1))

for i,sample in enumerate(samples):
  for j,character in enumerate(sample):
    index=token_index.get(character)
    print(index)
    results[i,j,index]=1
print(token_index,results)



**Creating word embeddings using layers.Embedding()**

In [0]:
from tensorflow.keras import layers,models,preprocessing
from tensorflow.keras.datasets import imdb
import numpy as np
maxlen=20
(Xtrain,Ytrain),(Xtest,Ytest)=imdb.load_data(num_words=1000)
Xtrain= preprocessing.sequence.pad_sequences(Xtrain,maxlen=maxlen)
Xtest=preprocessing.sequence.pad_sequences(Xtest,maxlen=maxlen)
model= models.Sequential()
model.add(layers.Embedding(1000,8,input_length=20))
model.add(layers.Flatten())
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])
model.fit(Xtrain,Ytrain,epochs=20)
model.evaluate(Xtest,Ytest)

Epoch 1/20
782/782 [==============================] - 13s 16ms/step - loss: 0.6207 - accuracy: 0.6470
Epoch 2/20
782/782 [==============================] - 13s 16ms/step - loss: 0.5157 - accuracy: 0.7434
Epoch 3/20
782/782 [==============================] - 12s 16ms/step - loss: 0.4962 - accuracy: 0.7590
Epoch 4/20
782/782 [==============================] - 13s 16ms/step - loss: 0.4833 - accuracy: 0.7678
Epoch 5/20
782/782 [==============================] - 13s 16ms/step - loss: 0.4704 - accuracy: 0.7780
Epoch 6/20
782/782 [==============================] - 13s 16ms/step - loss: 0.4588 - accuracy: 0.7833
Epoch 7/20
782/782 [==============================] - 13s 16ms/step - loss: 0.4494 - accuracy: 0.7916
Epoch 8/20
782/782 [==============================] - 12s 16ms/step - loss: 0.4408 - accuracy: 0.7956
Epoch 9/20
782/782 [==============================] - 13s 16ms/step - loss: 0.4340 - accuracy: 0.8016
Epoch 10/20
782/782 [==============================] - 12s 16ms/step - loss: 0.427

[0.6211338639259338, 0.7054799795150757]

**Classification using pretrained embeddings (GLoVE)**

In [0]:
#Download the raw text dataset
!wget --no-check-certificate "https://mng.bz/0tIo"
!unzip "0tIo"

#split the dataset into corresponding train and test files
%cd aclImdb
from os import listdir
train_text=[]
train_labels=[]
test_text=[]
test_labels=[]

for file_type in ['neg','pos']:
  for file in listdir('train/'+file_type):
    f=open('train/'+file_type+'/'+file)
    train_text.append(f.read())
    if file_type=='neg':
      train_labels.append(0)
    else:
      train_labels.append(1)


for file_type in ['neg','pos']:
  for file in listdir('test/'+file_type):
    f=open('test/'+file_type+'/'+file)
    test_text.append(f.read())
    if file_type=='neg':
      test_labels.append(0)
    else:
      test_labels.append(1)


In [0]:
#preprocessing the raw text
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen=100
maxsize=10000
train_size=200
val_size=10000

#breaking down sentences into tokens and converting them into vectores 
token=Tokenizer(num_words=maxsize)
token.fit_on_texts(train_text)
sequence=token.texts_to_sequences(train_text)
data=pad_sequences(sequence,maxlen)
word_index=token.word_index
train_labels=np.asarray(train_labels)

#shuffling the dataset, as it is present in a sequence (neg -> pos)
indices=np.arange(data.shape[0])
np.random.shuffle(indices)
train_text=data[indices]
train_labels=train_labels[indices]

#generating the training and validation set
val_text=train_text[train_size+1:val_size]
val_labels=train_labels[train_size+1:val_size]
train_text=train_text[:train_size]
train_labels=train_labels[:train_size]
%cd ..

#Downloading stanfords glove pretrained embedding model
!wget 'http://nlp.stanford.edu/data/glove.6B.zip'
!unzip 'glove.6B.zip'

#creating a dictinary with words and their corresponding vectors from the Glove model
from os import listdir
f=open('glove.6B.100d.txt')
embed={}
for line in f:
  sentence=line.split(" ")
  word=sentence[0]
  value=np.asarray(sentence[1:]).astype('float32')
  embed[word]=value


#Building the embedding matrix which will be input into the embedding layer

embedding_dim=100

embed_matrix=np.zeros((maxsize,embedding_dim))
for word,i in word_index.items():
  if i<maxsize:
    embed_vector=embed.get(word)
    if embed_vector is not None:
      embed_matrix[i]=embed_vector

#Building, training and testing the model
from tensorflow.keras import layers,models

model=models.Sequential()
model.add(layers.Embedding(maxsize,embedding_dim,input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid')) 
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model.layers[0].set_weights([embed_matrix])
model.layers[0].trainable=False
model.fit(train_text,train_labels,epochs=20,validation_data=(val_text,val_labels))


**Temperature forecasting problem**

In [0]:
#Download the data and unzip the files
!wget 'https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip'
!unzip jena_climate_2009_2016.csv.zip
rm jena_climate_2009_2016.csv.zip

#read the data from csv into numpy arrays
f=open('jena_climate_2009_2016.csv')
f=f.read()
lines=f.split('\n')
header=lines[0].split(',')
lines=lines[1:]
import numpy as np
float_data=np.zeros((len(lines),len(header)-1))
for i,line in enumerate(lines):
  values=[float(x) for x in line.split(",")[1:]]
  float_data[i,:]=values


#Normalizing the data using standard normalization
data_mean=float_data[:200000].mean(axis=0)
float_data-=data_mean
data_std=float_data[:200000].std(axis=0)
float_data/=data_std


#Creating a python generator function to format the data in a way, the neural nets can accept them
def datagen(data,lookback,delay,shuffle,min_index,max_index,batch_size,step):
  if max_index is None:
    max_index=len(data)-delay-1
  i=min_index+lookback
  while 1:
    if shuffle:
      row=np.random.randint(i,max_index,size=batch_size)
    else:

      if i+batch_size<=max_index:
        i=min_index+lookback
      row=np.arange(i,min(i+batch_size,max_index))
      i+=len(row)
      samples=np.zeros((len(row),lookback//step,data.shape[-1]))
      target=np.zeros((len(row),))

      for j,r in enumerate(row):
        indices=range(row[j]-lookback,row[j],step)
        samples[j]=data[indices]
        target[j]=data[row[j]+delay][1]
      yield samples,target

#Getting the train, test, and validation sets    
lookback=1440
step=6
batch_size=128
delay=144
train_gen=datagen(float_data,lookback,delay,shuffle=False,min_index=0,max_index=200000,batch_size=128,step=6)
test_gen=datagen(float_data,lookback,delay,shuffle=False,min_index=200000,max_index=300000,batch_size=128,step=6)
val_gen=datagen(float_data,lookback,delay,shuffle=False,min_index=300000,max_index=None,batch_size=128,step=6)


#Building our models: Stacked GRU with dropouts, Bidirectional RNN's
from tensorflow.keras import layers,models

model=models.Sequential()
#Using stacked gru's with recurrent dropout
# model.add(layers.GRU(32,dropout=0.1,recurrent_dropout=0.5,return_sequences=True,input_shape=(None,float_data.shape[-1])))
# model.add(layers.GRU(64,dropout=0.1,recurrent_dropout=0.5,activation='relu'))

#Using bidirectional rnn
#model.add(layers.Bidirectional(layers.GRU(32),input_shape=(None,float_data.shape[-1])))

model.add(layers.Dense(1))
model.compile(optimizer='rmsprop',loss='mae')
model.fit(train_gen,epochs=40,steps_per_epoch=500,validation_data=val_gen,validation_steps=200)


--2020-03-31 18:12:06--  https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.237.157
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.237.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13568290 (13M) [application/zip]
Saving to: ‘jena_climate_2009_2016.csv.zip’

jena_climate_2009_2 100%[===================>]  12.94M  18.3MB/s    in 0.7s    

2020-03-31 18:12:07 (18.3 MB/s) - ‘jena_climate_2009_2016.csv.zip’ saved [13568290/13568290]



**Chapter 8: Generative Deep Learning**

**Text generation with LSTM**

In [0]:
import tensorflow.keras as tfk
import numpy as np

path=tfk.utils.get_file('nietzsche.txt',origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text=open(path).read().lower()
print(len(text))
maxlen=60
step=3
sentences=[]
next_chars=[]
#for i in range(0,len(text)-maxlen,step):

600893


**Deep Dream**

In [0]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import backend as K
K.set_training_phase(0)
model=inception_v3.InceptionV3(weights='imagenet',include_top=False)


  

